# ***Data Encoding***
Not all machine learning methods can handle all sorts of features (expicitly the categorical features). Therefor features are encoded to ensure a smooth processing. This file encode train and test data.

In the following features are encoded by one of two methods:

Label_Encoding: Label Encoding transform values to become numbers between  0  and  𝑛−1 , where  𝑛  is the number of different labels Frequency_Encoding: Frequency Encoding is a special case of Label Encoding. Feature values are encoded based on there frequency. Transformed values are numbers between  0  and  𝑚 , where  𝑚  is the number of values with a frequency greater or equal than 2. The resulting encoded data is stored under ../data/train_endcoding.csv and ../data/test_encoding.csv.

# **import libararies**

In [ ]:
import pandas as pd
import numpy as np
%pip install dask[dataframe]
import dask.dataframe as dd
from sklearn import preprocessing
from tqdm import tqdm
import gc
import sys
import pickle
import warnings
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
if not sys.warnoptions:
    warnings.simplefilter("ignore")

     |████████████████████████████████| 92kB 4.6MB/s 
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=9cd65e967fdbd9d6641a53fc4c69a1855ffdfafdfc35e54e101747bd8a997b20
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built locket


In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [ ]:
def convert_types(df):
    # Convert data types to reduce memory
    for c in df:
        col_type = str(df[c].dtypes)
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        
        # Convert objects to category
        if col_type == 'object':
            df[c] = df[c].astype('category')
        
        # numerics
        elif col_type in numerics:
            c_min = df[c].min()
            c_max = df[c].max()
            if col_type[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[c] = df[c].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[c] = df[c].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[c] = df[c].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[c] = df[c].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[c] = df[c].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[c] = df[c].astype(np.float32)
                else:
                    df[c] = df[c].astype(np.float64)  
        
    return df

## ***import data***

In [ ]:
# target column
target = 'HasDetections'
# id from data set
data_id = 'MachineIdentifier'

In [ ]:
# load train data set
# use dask to load faster
file = '/content/drive/My Drive/microsoft/train_featureengineering.csv'
ddf = dd.read_csv(file, dtype = dtypes)
train = ddf.compute()

In [ ]:
 train.shape

(8921483, 78)

In [ ]:
train = convert_types(train)

In [ ]:
train_1=train[train['HasDetections']==1].sample(100000,random_state=42)
train_0=train[train['HasDetections']==0].sample(100000,random_state=42)
train_sample=pd.concat([train_1,train_0])

In [ ]:
train_sample.shape

(200000, 78)

# **Data Encoding**

In [ ]:
list_frequency_encoding = ['AppVersion',
 'AvSigVersion',
 'Census_OSVersion',
 'EngineVersion',
 'OsBuildLab']

In [ ]:
correct_feature_by_hand = ['AppVersion',
 'AvSigVersion',
 'Census_ActivationChannel',
 'Census_ChassisTypeName',
 'Census_DeviceFamily',
 'Census_FlightRing',
 'Census_GenuineStateName',
 'Census_MDC2FormFactor',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSEdition',
 'Census_OSInstallTypeName',
 'Census_OSSkuName',
 'Census_OSVersion',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_PowerPlatformRoleName',
 'Census_PrimaryDiskTypeName',
 'EngineVersion',
 'MachineIdentifier',
 'OsBuildLab',
 'OsPlatformSubRelease',
 'OsVer',
 'Platform',
 'Processor',
 'SkuEdition',
 'SmartScreen']

In [ ]:

# new features from feature engineering
new_features_labelencode = ['monitor_dims', 'SmartScreen_AVProductsInstalled']

In [ ]:
list_label_encoding = list(set(correct_feature_by_hand)-set(list_frequency_encoding))
list_label_encoding.remove(data_id)
list_label_encoding.extend(new_features_labelencode)

In [ ]:
list_label_encoding

['OsVer',
 'Census_GenuineStateName',
 'Census_ChassisTypeName',
 'SmartScreen',
 'Census_OSInstallTypeName',
 'Census_OSEdition',
 'Census_MDC2FormFactor',
 'Census_FlightRing',
 'Census_OSWUAutoUpdateOptionsName',
 'OsPlatformSubRelease',
 'Census_PowerPlatformRoleName',
 'Census_OSArchitecture',
 'Census_DeviceFamily',
 'Census_OSBranch',
 'Census_OSSkuName',
 'Census_PrimaryDiskTypeName',
 'Census_ActivationChannel',
 'Platform',
 'Processor',
 'SkuEdition',
 'monitor_dims',
 'SmartScreen_AVProductsInstalled']

In [ ]:
def frequency_encoding(feature):
    # Count the number of values of each feature and reset the indices
    t = train_sample[feature].value_counts().reset_index()
    # Building up a new index (old index is set by default to 'level_0')
    t = t.reset_index()
    # Set the old index 'level_0' for all values, which only occur once, to NaN
    t.loc[t[feature] == 1, 'level_0'] = np.nan
    # Reset the original index (= the value name) as index
    t.set_index('index', inplace=True)
    # return the number of values , which occur two or more times, +1
    max_label = t['level_0'].max() + 1
    # fill all nan-values to max_label
    t.fillna(max_label, inplace=True)
    return t.to_dict()['level_0']

In [ ]:
# Creating a dictionary for storing the encoder
enc_dict = dict();

In [ ]:
# Encode all the features in 'list_frequency_encoding'
for feature in tqdm(list_frequency_encoding):
    freq_enc_dict = frequency_encoding(feature)
    train_sample[feature] = train_sample[feature].map(lambda x: freq_enc_dict.get(x, np.nan))
    train_sample[feature] = train_sample[feature].astype('int64')    
    # Save the freq_enc_dict
    enc_dict[feature] = freq_enc_dict

100%|██████████| 5/5 [00:00<00:00, 74.89it/s]


In [ ]:
train_sample.shape

(200000, 78)

In [ ]:
train_sample=pd.get_dummies(train_sample,columns=list_label_encoding)

In [ ]:
train_sample.shape

(200000, 10412)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test = df[df[target].isnull()]
test.shape

In [ ]:
train=df[df[target].notnull]
train.shape

In [ ]:
# Save the encoded data-files
train_sample.to_csv('/content/drive/My Drive/microsoft/train_sample_onehotencoding_sample.csv', index = False)

### **Dimensionality Reduction**
Since after one hot encoding, the dimension of the design matrix becomes huge such that training our model becomes computationally expensive. We use only a set of selected features for training our model. In stage 1 we use data_vars fumction to give us an initial estimate of the importance of features. After that we threshold the importance and remove the features which lie below a certain threshold. In second step we use iterate_vif function to remove highly coorelated indepedent variables

**Taking the sample of population with equal distribution of HasDetections**

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
train_sample=pd.read_csv('/content/drive/My Drive/microsoft/train_sample_onehotencoding_sample.csv')

In [ ]:
train_sample.shape

(200000, 10412)

In [ ]:
# target column
target = 'HasDetections'
# id from data set
data_id = 'MachineIdentifier'

In [ ]:
y_train=train_sample[target]
X_train=train_sample.drop([target,data_id],axis=1)

In [ ]:
max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [ ]:
final_iv, IV = data_vars(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
features = list(IV[(IV['IV'] >= 0.01) & (IV['IV'] <= 0.8)]['VAR_NAME'])
X2 = X_train[features]

In [ ]:
X2.shape

(200000, 27)

In [ ]:
def iterate_vif(df, vif_threshold=5, max_vif=6):
  count = 0
  while max_vif > vif_threshold:
    count += 1
    print("Iteration # "+str(count))
    vif = pd.DataFrame()
    vif["VIFactor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif["features"] = df.columns
    
    if vif['VIFactor'].max() > vif_threshold:
      print('Removing %s with VIF of %f' % (vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], vif['VIFactor'].max()))
      df = df.drop(vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], axis=1)
      max_vif = vif['VIFactor'].max()
    else:
        print('Complete')
        return df, vif.sort_values('VIFactor')
 
final_df, final_vif = iterate_vif(X2)

Iteration # 1


/usr/local/lib/python3.6/dist-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Removing Census_OSArchitecture_amd64 with VIF of inf
Iteration # 2
Removing Processor_x64 with VIF of 74.859053
Iteration # 3
Removing SmartScreen_existsnotset with VIF of 61.418731
Iteration # 4
Removing Processor_x86 with VIF of 57.281494
Iteration # 5
Removing AVProductsEnabled with VIF of 34.508172
Iteration # 6
Removing AVProductsInstalled with VIF of 21.370763
Iteration # 7
Removing IsProtected with VIF of 9.694294
Iteration # 8
Removing Census_PowerPlatformRoleName_slate with VIF of 6.723769
Iteration # 9
Complete


In [ ]:
X1 = X2._get_numeric_data()
X_train, final_vif = iterate_vif(X1)

Iteration # 1


/usr/local/lib/python3.6/dist-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Removing Census_OSArchitecture_amd64 with VIF of inf
Iteration # 2
Removing Processor_x64 with VIF of 74.859053
Iteration # 3
Removing SmartScreen_existsnotset with VIF of 61.418731
Iteration # 4
Removing Processor_x86 with VIF of 57.281494
Iteration # 5
Removing AVProductsEnabled with VIF of 34.508172
Iteration # 6
Removing AVProductsInstalled with VIF of 21.370763
Iteration # 7
Removing IsProtected with VIF of 9.694294
Iteration # 8
Removing Census_PowerPlatformRoleName_slate with VIF of 6.723769
Iteration # 9
Complete


In [ ]:
X_train.shape

(200000, 19)

In [ ]:
X_test=test[X_train.columns]
X_test.shape

(7853253, 12)

In [ ]:
X_train.to_csv('/content/drive/My Drive/label_encoding/X_train_onehotencoding.csv', index = False)
y_train.to_csv('/content/drive/My Drive/label_encoding/y_train_onehotencoding.csv', index = False)
#X_test.to_csv('//content/drive/My Drive/label_encoding/X_test.csv', index = False)

In [ ]:
X_train.columns

Index(['AVProductStatesIdentifier', 'AppVersion',
       'Census_IsAlwaysOnAlwaysConnectedCapable', 'Census_IsVirtualDevice',
       'Census_MDC2FormFactor_tablet', 'Census_OSArchitecture_x86',
       'Census_OSVersion', 'Census_TotalPhysicalRAM', 'EngineVersion',
       'OsBuildLab', 'ProcessorCoreCount_DisplaySizeInInches',
       'SmartScreen_AVProductsInstalled_existsnotset1.0',
       'SmartScreen_AVProductsInstalled_existsnotset2.0',
       'SmartScreen_AVProductsInstalled_requireadmin2.0',
       'SmartScreen_AVProductsInstalled_requireadmin3.0',
       'SmartScreen_AVProductsInstalled_unknown2.0',
       'SmartScreen_requireadmin', 'Wdft_IsGamer',
       'monitor_dims_1280.0*800.0'],
      dtype='object')